This tutorial demonstrates how to __read a SEG-Y data set using a customized trace header definition table__. The test data is a small SEG-Y file containing three traces with special trace headers such as a shot point line number, a receiver point line number, or a floating datum elevation.

Let's start by importing some packages (including the __seisio__ package itself) and setting up the logging config.

In [1]:
import logging
import numpy as np
import sys
sys.path.insert(1, '../')
import seisio

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)
log = logging.getLogger("main")

We would like to read the file called "three_traces.sgy" located in the data directory. In order to do so, we use a customized trace header definition table. We can get a trace header definition template using the *segy_thdef_template()* function of the __seisio__ package, we just have to provide the path and name under which the file should be stored.

In [2]:
seisio.segy_thdef_template("./custom_segy_thdef.json")

Now we could edit this file to customize it for the SEG-Y file at hand. A pre-edited version is available under the name *three_traces_thdef.json* in the data directory. Here, the standard headers that involve the string "cdp" were renamed "cmp", and there are some special definitions in byte locations 197 onwards. Please note that we could also read, for instance, floating-point values (32-bit or 64-bit, symbol "f" or "d", respectively) or other types of entries not usually present in the SEG-Y standard.

In [3]:
infile = "data/three_traces.sgy"
sio = seisio.input(infile, thdef="data/three_traces_thdef.json")

INFO: Input file: data/three_traces.sgy
INFO: SEG-Y textual file header encoding looks to be 'EBCDIC' (best guess).
INFO: Input file endianess looks to be '>' (best guess).
INFO: SEG-Y revision: original SEG-Y conforming to 1975 standard.
INFO: Data sample format: 4-byte IBM floating-point.
INFO: Number of additional textual header records: 0.
INFO: Number of trailer stanza records: 0.
INFO: SEG-Y trace header extension 1 is not present.
INFO: Number of user-defined trace headers: 0
INFO: Byte offset of first trace relative to start of file: 3600 bytes.
INFO: Number of samples per data trace: 1334.
INFO: Sampling interval: 3000 (unit as per SEG-Y standard).
INFO: Number of data traces in file: 3.
INFO: Delay (on first trace): 0 (unit as per SEG-Y standard).


If you get an error about titles already being used when creating the custom data type for Numpy, please ensure that the description fields in the trace header definition file (json) contain unique entries. Now let's read the trace headers from disk and determine some trace header statistics:

In [4]:
thstat = sio.log_thstat()

INFO: Calculating trace header statistics.
INFO: Reading all 3 trace headers from disk...
INFO: Reading all headers took 0.000 seconds.
INFO: Summary of trace header statistics (zeros excluded):
+---------+------------------+------------------+------------------+---------+------------------+------------------+
|         |              min |              max |             mean |     std |              25% |              75% |
|---------+------------------+------------------+------------------+---------+------------------+------------------|
| fldr    | 200001           | 200001           | 200001           |   0     | 200001           | 200001           |
| tracf   |      1           |      3           |      2           |   1     |      1.5         |      2.5         |
| cmp     |      1.37501e+08 |      1.37501e+08 |      1.37501e+08 |   1     |      1.37501e+08 |      1.37501e+08 |
| trid    |      1           |      1           |      1           |   0     |      1           |      

As can clearly be seen, there are no trace headers with the string "cdp...", all those headers are called "cmp..." as expected, and we have a few custom headers called "fldatel", "binel", "spline", "sp", "rpline", "rp", "rpidx", "spiid", "rpiid" and "spidx". Those were defined in the customized JSON trace header definition table. Please note that the customized header definition is not allowed to have gaps and/or overlaps in byte positions.

The custom trace headers can be accessed as usual:

In [5]:
dataset = sio.read_dataset()
fldatel = dataset["fldatel"]
scalco = dataset["scalco"]
float_dat = fldatel/np.abs(scalco)
log.info("Floating datum elevations in meters: %s", float_dat)

INFO: Reading entire file (3 traces) from disk...
INFO: Reading all traces took 0.000 seconds.
INFO: Converting IBM floats to IEEE floats.
INFO: Floating datum elevations in meters: [106.76 106.97 107.18]


If you write the data back to disk in SEG-Y format, you have to use the same customized trace header definition table on output, otherwise the custom trace headers get lost. Or, alternatively, you have to remap the headers from their customized names to trace header names and types available in the standard SEG-Y trace header definition table.